In [1]:
import pandas as pd
import numpy as np

import time

from datasource import * 

In [2]:
def get_new_cycle_records(source_df, target_cycle):
    ret = []
    
    if source_df is None or len(source_df) < 2:
        return None
    
    source_len = len(source_df)
    source_cycle = source_df.iloc[-1]['date'] - source_df.iloc[-2]['date']

    # source_cycle: pd.Timedelta
    # ms = source_cycle.total_seconds() * ms
    source_cycle =  source_cycle.total_seconds() * 1000

    if target_cycle % source_cycle != 0:
        raise ValueError("target_cycle is not an integral multiple of source_cycle.")
    
    if (1000 * 60 * 60) % target_cycle != 0 and (1000 * 60 * 60 * 24) % target_cycle != 0:
        raise ValueError("target_cycle cannot complete the cycle.")
    
    multiple = target_cycle / source_cycle
    
    is_begin = False
    count = 0
    high = 0
    low = 0
    open = 0
    close = 0
    date = 0
    vol = 0
    
    for i in range(source_len):
        # source_df.iloc[i]['date']: pd.Timedelta
        # ms = source_df.iloc[i]['date'].timestamp() * 1000
        if ((1000 * 60 * 60 * 24) - (source_df.iloc[i]['date'].timestamp() * 1000) % (1000 * 60 * 60 * 24) +
            (pd.to_datetime('now').tz_localize('UTC').tz_convert('Asia/Shanghai').utcoffset().total_seconds() * 1000 * 60)) % target_cycle == 0:
            is_begin = True
        
        if is_begin:
            if count == 0:
                date = source_df.iloc[i]['date']
                open = source_df.iloc[i]['open']
                close = source_df.iloc[i]['close']
                high = source_df.iloc[i]['high']
                low = source_df.iloc[i]['low']
                vol = source_df.iloc[i]['volume']
                count += 1
            elif count < multiple:
                close = source_df.iloc[i]['close']
                high = max(high, source_df.iloc[i]['high'])
                low = min(low, source_df.iloc[i]['low'])
                vol += source_df.iloc[i]['volume']
                count += 1
            
            if count == multiple or i == source_len - 1:
                ret.append({
                    'date': date,
                    'open': open,
                    'close': close,
                    'high': high,
                    'low': low,
                    'volume': vol,
                })
                count = 0
    columns = [
        "date",
        "open",
        "close",
        "high",
        "low",
        "volume"
    ]
    return pd.DataFrame(ret, columns=columns)

In [3]:
code_list = [
    ['518880.XSHG','518880.XSHG'], #黄金ETF  518880
    ['513100.XSHG','513100.XSHG'], #纳指ETF  513100
    ['399006.XSHE','159915.XSHE'], #创业板 399006 159915 159952 512900
    # ['513050.XSHG','513050.XSHG'], #中概互联 ETF  
    # ['399673.XSHE','399673.XSHE'], #创业板50 399673 159949
    ['000016.XSHG','000016.XSHG'], #上证50 000016 510050 510710 510850
    # ['000300.XSHG','510300.XSHG'], #沪深300
    # ['000905.XSHG','510500.XSHG'] #中证500
    # ['399975.XSHE','399975.XSHE'], #证劵公司 399975 512880 512000 512900
    # ['399987.XSHE','399987.XSHE'], #中证酒
    # ['399932.XSHE','159928.XSHE'], #消费ETF
    # ['000913.XSHG','000913.XSHG'], #300医药 000913 399913 000913
    # ['515000.XSHG','515000.XSHG'], #科技 515000
    # ['000015.XSHG','510880.XSHG'], #红利ETF
]

code_list_ = [l[1].split(".")[0] for l in code_list]

In [4]:
def get_source_df():
    fund_etf_hist_min_em_df = fund_etf_hist_min_em(
        symbol = code_list_[2],
        period="1",
        adjust="qfq",
        start_date="2024-01-02 09:30:00",
        end_date="2024-01-08 15:00:00",
    )

    fund_etf_hist_min_em_df.columns = [
        "date",
        "open",
        "close",
        "high",
        "low",
        "volume",
        "amount",
        "price",
    ]

    source_df = fund_etf_hist_min_em_df
    source_df["date"] = pd.to_datetime(source_df["date"])
    # source_df = fund_etf_hist_min_em_df.set_index(["date"], drop=True, inplace=False)
    print(source_df)
    return source_df

In [5]:
source_df = get_source_df()

                   date  open  close   high    low  volume      amount   price
0   2024-01-03 09:30:00   0.0  1.802  1.802  1.802   94990  17117198.0  1.8020
1   2024-01-03 09:31:00   0.0  1.800  1.803  1.800  152645  27492771.0  1.8014
2   2024-01-03 09:32:00   0.0  1.802  1.803  1.800   85451  15386569.0  1.8012
3   2024-01-03 09:33:00   0.0  1.805  1.806  1.803  146436  26423795.0  1.8022
4   2024-01-03 09:34:00   0.0  1.807  1.807  1.804   50899   9193101.0  1.8026
..                  ...   ...    ...    ...    ...     ...         ...     ...
959 2024-01-08 14:56:00   0.0  1.699  1.700  1.699  126421  21482267.0  1.7130
960 2024-01-08 14:57:00   0.0  1.701  1.701  1.698  193750  32921698.0  1.7128
961 2024-01-08 14:58:00   0.0  1.700  1.700  1.700     573     97410.0  1.7128
962 2024-01-08 14:59:00   0.0  1.700  1.700  1.700       0         0.0  1.7128
963 2024-01-08 15:00:00   0.0  1.699  1.699  1.699  129431  21990326.0  1.7127

[964 rows x 8 columns]


In [6]:
source_df

,date,open,close,high,low,volume,amount,price
0,2024-01-03 09:30:00,0.0,1.802,1.802,1.802,94990,17117198.0,1.8020
1,2024-01-03 09:31:00,0.0,1.800,1.803,1.800,152645,27492771.0,1.8014
2,2024-01-03 09:32:00,0.0,1.802,1.803,1.800,85451,15386569.0,1.8012
3,2024-01-03 09:33:00,0.0,1.805,1.806,1.803,146436,26423795.0,1.8022
4,2024-01-03 09:34:00,0.0,1.807,1.807,1.804,50899,9193101.0,1.8026
...,...,...,...,...,...,...,...,...
959,2024-01-08 14:56:00,0.0,1.699,1.700,1.699,126421,21482267.0,1.7130
960,2024-01-08 14:57:00,0.0,1.701,1.701,1.698,193750,32921698.0,1.7128
961,2024-01-08 14:58:00,0.0,1.700,1.700,1.700,573,97410.0,1.7128
962,2024-01-08 14:59:00,0.0,1.700,1.700,1.700,0,0.0,1.7128


In [7]:
def case_1():
    """
    # Example usage:
    # Assuming source_df is a pandas DataFrame with columns: 'high', 'low', 'open', 'close', 'Time', 'volume'
    # target_cycle is the desired target cycle in milliseconds

    # source_df = pd.DataFrame(...)  # Replace ... with your data
    # target_cycle = 1000 * 60 * 60 * 4  # 4 hours in milliseconds

    # new_cycle_df = get_new_cycle_records(source_df, target_cycle)
    # print(new_cycle_df)

    """ 

    # while True:
    #     r = exchange.get_records()  # 原始数据，作为合成K线的基础K线数据，例如要合成4小时K线，可以用1小时K线作为原始数据。
    #     r2 = get_new_cycle_records(r, 1000 * 60 * 60 * 4)  # 通过 GetNewCycleRecords 函数 传入 原始K线数据 r , 和目标周期， 1000 * 60 * 60 * 4 即 目标合成的周期 是4小时K线数据。

    #     # 在这里插入你的画图逻辑

    #     time.sleep(1)  # 每次循环间隔 1 秒，防止访问K线接口获取数据过于频繁，导致交易所限制

    new_df = get_new_cycle_records(source_df=source_df, target_cycle=1000 * 60 * 3)
    # print(new_df)

    return new_df

def case_2():
    new_df = get_new_cycle_records(source_df=source_df, target_cycle=1000 * 60 * 5)
    # print(new_df)
    return new_df

def case_3():
    new_df = get_new_cycle_records(source_df=source_df, target_cycle=1000 * 60 * 15)
    # print(new_df)
    return new_df

In [8]:
new_1_df = case_1()

In [9]:
new_1_df

,date,open,close,high,low,volume
0,2024-01-03 09:30:00,0.0,1.802,1.803,1.800,333086
1,2024-01-03 09:33:00,0.0,1.807,1.807,1.803,277085
2,2024-01-03 09:36:00,0.0,1.808,1.809,1.806,144588
3,2024-01-03 09:39:00,0.0,1.802,1.808,1.801,113374
4,2024-01-03 09:42:00,0.0,1.802,1.803,1.800,99831
...,...,...,...,...,...,...
317,2024-01-08 14:48:00,0.0,1.700,1.701,1.699,212722
318,2024-01-08 14:51:00,0.0,1.701,1.701,1.700,194826
319,2024-01-08 14:54:00,0.0,1.699,1.701,1.698,274413
320,2024-01-08 14:57:00,0.0,1.700,1.701,1.698,194323


In [10]:
new_2_df = case_2()

In [11]:
new_2_df

,date,open,close,high,low,volume
0,2024-01-03 09:30:00,0.0,1.807,1.807,1.800,530421
1,2024-01-03 09:35:00,0.0,1.806,1.809,1.805,258365
2,2024-01-03 09:40:00,0.0,1.802,1.806,1.800,179178
3,2024-01-03 09:45:00,0.0,1.803,1.804,1.800,150562
4,2024-01-03 09:50:00,0.0,1.804,1.805,1.801,169016
...,...,...,...,...,...,...
188,2024-01-08 14:37:00,0.0,1.700,1.702,1.698,607744
189,2024-01-08 14:42:00,0.0,1.700,1.701,1.698,587758
190,2024-01-08 14:47:00,0.0,1.701,1.701,1.699,340966
191,2024-01-08 14:52:00,0.0,1.699,1.701,1.698,420665


In [12]:
new_3_df = case_3()

In [13]:
new_3_df

,date,open,close,high,low,volume
0,2024-01-03 09:30:00,0.0,1.802,1.809,1.800,967964
1,2024-01-03 09:45:00,0.0,1.799,1.805,1.799,502869
2,2024-01-03 10:00:00,0.0,1.793,1.800,1.791,754958
3,2024-01-03 10:15:00,0.0,1.795,1.796,1.791,407137
4,2024-01-03 10:30:00,0.0,1.794,1.798,1.793,286178
...,...,...,...,...,...,...
60,2024-01-08 13:57:00,0.0,1.707,1.716,1.702,683746
61,2024-01-08 14:12:00,0.0,1.710,1.710,1.704,343122
62,2024-01-08 14:27:00,0.0,1.700,1.711,1.698,1439396
63,2024-01-08 14:42:00,0.0,1.699,1.701,1.698,1349389
